# Analysis Development
 - Given the user has access to Spotify, this script will return a simple plot of their liked songs grouped by day and number of songs liked per day
 - Additional updated research will be appended in here

##### Import the modules

In [1]:
import numpy as np
import pandas as pd

import spotipy
from spotipy import SpotifyOAuth

##### Client ID and Secret are passed from an external source

In [2]:
import settings

In [3]:
# 
spotify_client_id = settings.SPOTIFY_CLIENT_ID
spotify_client_secret = settings.SPOTIFY_CLIENT_SECRET
spotify_redirect_uri = "http://localhost:5000/callback"

In [4]:
scope =  'user-read-private playlist-read-private playlist-read-collaborative playlist-modify-public playlist-modify-private'
auth_manager = SpotifyOAuth(
                            scope = scope, 
                            client_id = spotify_client_id,
                            client_secret = spotify_client_secret,
                            redirect_uri = spotify_redirect_uri
                            )

sp = spotipy.Spotify(auth_manager = auth_manager)

In [6]:
sp.current_user()